# Homework 6, Exercise 4

## Lab Instructions
All your answers for this exercise should be written in this notebook. You shouldn't need to write or modify any other files.

**You should execute every block of code to not miss any dependency.**

*This exercise was developed by Maximilian Hüttenrauch for the KIT Cognitive Systems Lecture, July 2020.*

In [1]:
from ex4.quad_link import QuadLink
import numpy as np
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: MacOSX


Exercise 4a) Jacobian Transpose DIK (you can change alpha values for different behavior)

In [2]:
def jacobian_transpose_dik(ql, alpha=2):
    """
    Computes the Jacobian Transposed DIK
    :param ql: A QuadLink object
    :param alpha: "learning rate"
    :return: Desired joint velocities
    """
    jac = ql.get_jacobian()
    err = ql.get_error()

    q_dot_des = alpha * jac.T @ err


    assert q_dot_des.shape == (4, 1)
    return q_dot_des

Exercise 4b) Jacobian Pseudo Inverse DIK (you can change alpha values for different behavior)

In [3]:
def jacobian_pinv_dik(ql, alpha=10):
    """
    Computes the Jacobian Pseudo Inverse DIK
    :param ql: A QuadLink object
    :param alpha: "learning rate"
    :return: Desired joint velocities
    """
    jac = ql.get_jacobian()
    err = ql.get_error()

    q_dot_des = alpha * jac.T @ np.linalg.solve(jac @ jac.T, err)

    assert q_dot_des.shape == (4, 1)
    return q_dot_des

Exercise 4c) Jacobian Damped Pseudo Inverse DIK (you can change alpha and lambda values for different behavior)

In [4]:
def jacobian_damped_pinv_dik(ql, alpha=10, lamb=1e-2):
    """
    Computes the Jacobian Damped Pseudo Inverse DIK
    :param ql: A QuadLink object
    :param alpha: "learning rate"
    :param lamb: Damping factor
    :return:
    """
    jac = ql.get_jacobian()
    err = ql.get_error()

    q_dot_des = alpha * jac.T @ np.linalg.solve(jac @ jac.T + lamb * np.eye(2), err)

    assert q_dot_des.shape == (4, 1)
    return q_dot_des

zero_vel and random_vel return a 0 velocity or random velocity and are only there as examples.

In [5]:
def zero_vel():
    return np.zeros(shape=(4, 1))

In [6]:
def random_vel():
    return 10 * np.random.standard_normal(size=(4, 1))

In [7]:
def get_q_des(q_dot_des, ql):
    return ql.q + ql.dt * q_dot_des

Exercise 4d) PD-controller (you can change gains for different behavior)

In [8]:
def pd_control(q_des, q_dot_des, ql, kp=20, kd=20):
    """
    PD-controller
    :param q_des: Desired joint configuration
    :param q_dot_des: Desired joint velocity
    :param ql: A QuadLink object
    :param kp: Proportional gain
    :param kd: Derivative gain
    :return:
    """
    u = kp * (q_des - ql.q) + kd * (q_dot_des - ql.q_dot)
    assert u.shape == (4, 1)
    return u

In [9]:
# No changes necessary in here
def simulate(ql, dik="zero", n_steps=200, render=True):
    """
    Run the quad link simulation for n_steps time steps with differential inverse kinematics dik
    :param ql: A QuadLink object
    :param dik: The type of differential inverse kinematics (string)
    :param n_steps: Number of time steps
    :param render: If true, show rendering. If false, run without rendering.
    :return:
    """

    ql.reset()

    qs = []
    q_dots = []
    costs = []
    qs.append(ql.q)
    q_dots.append(ql.q_dot)

    if render:
        ql.render()

    for i in range(n_steps):

        if dik == "zero":
            q_dot_des = zero_vel()
        elif dik == "random":
            q_dot_des = random_vel()
        elif dik == "jac_t":
            q_dot_des = jacobian_transpose_dik(ql)
        elif dik == "jac_pinv":
            q_dot_des = jacobian_pinv_dik(ql)
        elif dik == "jac_damped_pinv":
            q_dot_des = jacobian_damped_pinv_dik(ql)
        else:
            raise ValueError("unknown dik")

        q_des = get_q_des(q_dot_des, ql)

        u = pd_control(q_des, q_dot_des, ql)

        cost = ql.step(u)

        qs.append(ql.q)
        q_dots.append(ql.q_dot)
        costs.append(cost)

        if render:
            ql.render()

    return qs, q_dots, costs


In [10]:
# Try different initializations here
# q_init = np.zeros(shape=(4, 1))
q_init = 1 * np.random.standard_normal(size=(4, 1))
# q_init = 0.1 * np.ones(shape=(4, 1))
q_dot_init = np.zeros(shape=(4, 1))

# Try different goal states here
x_goal = np.array((-1, -1))

quad_link = QuadLink(q_init, q_dot_init, x_goal)

In [11]:
"Run a quad link simulation, uncomment dik_type for different calculations of \dot{q}_des"
# dik_type = "zero"
# dik_type = "random"
dik_type = "jac_t"
# dik_type = "jac_pinv"
# dik_type = "jac_damped_pinv"

traj_q, traj_q_dot, costs = simulate(quad_link, dik=dik_type, render=True)

### Exercise 4e) Singularities
Look at the trajectories for a multiple zero initialization of q_init.

Each line in the first plot represents the trajectory of a joint angle.

Each line in the second plot represents the trajectory of a joint velocity.

In [22]:
fig, ax = plt.subplots(2)
ax[0].plot(np.hstack(traj_q).T);
ax[1].plot(np.hstack(traj_q_dot).T);

In [ ]:
from solution_checker import check_ex4
check_ex4(get_q_des, jacobian_transpose_dik, jacobian_pinv_dik, jacobian_damped_pinv_dik, pd_control)

